In [62]:
using BenchmarkTools: @btime

## HO Basis functions

In [5]:
struct HO
    n::Float64
    factors::Vector{Float64}
    parity::Float64
    ω::Float64

    function HO(n, ω)
        factors = Float64[]
        
        scaling = sqrt( factorial(n) / 2^n ) * (ω / π)^0.25
        ξfac = √ω
        
        for m in n÷2:-1:0
            factor = scaling * (-1)^m / (factorial(m) * factorial(n - 2m)) * 2^(n - 2m) * ξfac^(n - 2m)
            push!(factors, factor)
        end
        return new(n, factors, n % 2, ω)
    end
end

In [6]:
function ho(x, ho)
    result = zero(x)
    x2 = x^2
    xm = (x * ho.parity) + (1 - ho.parity) # xm = 1 if parity == 1 else xm = x
    for factor in ho.factors
        result += factor * xm
        xm *= x2
    end
    result = result * exp(-ho.ω * x^2 / 2)
    return result
end

ho (generic function with 1 method)

In [7]:
function potential(x, basis)
    return 0.5 * basis.ω^2 * x^2
end

potential (generic function with 1 method)

In [8]:
function E_n(basis)
    return (basis.n + 1/2) * basis.ω
end

E_n (generic function with 1 method)

## h

$$
\begin{equation}
\begin{aligned}
h_{\mu \nu} &= \langle \psi_\mu | \hat{h} | \psi_\nu \rangle \\
&= \langle \psi_\mu | \epsilon_\nu | \psi_\nu \rangle \\
&= \epsilon_\nu \delta_{\mu , \nu}
\end{aligned}
\end{equation}
$$

In [9]:
import LinearAlgebra as la

In [346]:
basis = [HO(n, 0.25) for n in 0:1];

In [347]:
h = la.Diagonal([E_n(i) for i in basis])

2×2 LinearAlgebra.Diagonal{Float64, Vector{Float64}}:
 0.125   ⋅ 
  ⋅     0.375

In [348]:
Array(h)

2×2 Matrix{Float64}:
 0.125  0.0
 0.0    0.375

## u

$$
\begin{equation}
\begin{aligned}
u_{\mu \nu} &= \sum_{\kappa, \lambda = 1}^l D_{\lambda, \kappa} ( \langle \psi_\mu \psi_\kappa | \hat{u} | \psi_\nu \psi_\lambda \rangle - \langle \psi_\mu \psi_\kappa | \hat{u} | \psi_\lambda \psi_\nu \rangle ) \\
D_{\lambda, \kappa} &= \sum_{i=1}^n C^*_{\kappa i} C_{\lambda i} \\
u_{\nu \lambda}^{\mu \kappa} &= \langle \psi_\mu \psi_\kappa | \hat{u} | \psi_\nu \psi_\lambda \rangle = \int \int dx_1 dx_2 \psi_\mu(x_1) \psi_\kappa(x_2) u(x_1, x_2) \psi_\nu(x_1) \psi_\lambda(x_2)
\end{aligned}
\end{equation}
$$

For every value of x1, compute the inner integral over x2.

Then use the values for the inner integral to compute the integral over x1.

In [349]:
[i for i in range(-10, stop=10, length=2001)];

In [350]:
function _trapz(f_vals, grid)
    Δx = grid[2] - grid[1]
    
    val = 0.0
    for i in 2:length(f_vals)
        val += f_vals[i - 1] + f_vals[i]
    end

    return 0.5 * val * Δx
end;

In [358]:
l = 2
basis = [HO(i, 0.25) for i in 0:l-1]
grid = range(-10, stop=10, length=201)
spfs = [ho.(grid, (ψ_i,)) for ψ_i in basis];

In [359]:
function inner_ints(l, spfs, grid)
    # Inner integrals
    inner_int = zeros(l, l, 201)
    f_vals = zero(grid)
    for κ in 1:l
        for λ in 1:l
            for (xi, x1) in enumerate(grid)
                f_vals .= conj.(spfs[κ]) .* (1 ./ sqrt.( (x1 .- grid).^2 .+ 0.25^2)) .* spfs[λ]
                inner_int[κ, λ, xi] = _trapz(f_vals, grid)
            end
        end
    end
    return inner_int
end

inner_ints (generic function with 1 method)

In [360]:
@time inner_int = inner_ints(l, spfs, grid);

  0.082282 seconds (279.14 k allocations: 14.967 MiB, 99.08% compilation time)


In [368]:
inner_int[:, :, 50]

2×2 Matrix{Float64}:
  0.217317   -0.0780608
 -0.0780608   0.283477

In [396]:
function outer_int(l, spfs, grid, inner_ints)
    outer_int = zeros(l, l, l, l)
    f_vals = zero(grid)
    for κ in 1:l
        for λ in 1:l
            for μ in 1:l
                for ν in 1:l
                    @views f_vals .= conj.(spfs[μ]) .* inner_ints[κ, λ, :] .* spfs[ν]
                    outer_int[μ, κ, ν, λ] = _trapz(f_vals, grid)
                end
            end
        end
    end
    return outer_int
end

outer_int (generic function with 1 method)

In [397]:
@time outer = outer_int(l, spfs, grid, inner_int);

  0.090518 seconds (54.32 k allocations: 2.774 MiB, 99.96% compilation time)


In [385]:
outer[:, :, 1, 1]

2×2 Matrix{Float64}:
 1.13365      6.54244e-17
 2.15095e-17  0.371012

In [192]:
outer .- permutedims(outer, [1, 2, 4, 3]);

In [339]:
function add_spin_u(u_old)
    """
    When we make a spin-up and spin-down duplicate of each basis function, we get many new integrals to compute
    between these new basis functions. However, most of these integrals will be zero due to opposite spins between
    the basis functions, or they will be the same as the old integrals, if the spins align.
    
    We loop over the latter two indices in our two-body integral, ν and λ. The matrix of elements at the indeces
    u_new[:, :, ν, λ] then correspond to integrals where the first two basis functions in the integral have spins
    alternating up and down.
    
    'up-up'   'up-down'   up-up   ...
    'down-up' 'down-down' down-up ...
     up-up     up-down    up-up   ...
     ...       ...        ...     ...
    
    This 2x2 pattern of spins (that we have marked with '') repeats, and all elements in the 2x2 pattern has the same
    set of spatial functions. But only one element will be non-zero, the one that has the same spins as the basis
    functions numbered ν and λ.
    
    The code below finds which one of these elements in the 2x2 pattern will be non-zero, and then uses this to turn
    the two-body-integrals without spin into the two-body-integrals with spin.
    """
    l = size(outer)[1] * 2
    u_new = zeros((l, l, l, l))
    
    for ν in 1:l
        for λ in 1:l
            if (ν % 2 == 1)     # --- UP ---
                if (λ % 2 == 1) # UP - UP
                    dir = [1 0; 0 0]
                else            # UP - DOWN
                    dir = [0 1; 0 0]
                end
            else                # --- DOWN ---
                if (λ % 2 == 1) # DOWN - UP
                    dir = [0 0; 1 0]
                else            # DOWN - DOWN
                    dir = [0 0; 0 1]
                end
            end
            
            ν_old = Int( ceil(ν / 2) )
            λ_old = Int( ceil(λ / 2) )
            u_new[:, :, ν, λ] .= kron(u_old[:, :, ν_old, λ_old], dir)
        end
    end
    return u_new
end

add_spin_u (generic function with 1 method)

In [338]:
u_new = add_spin_u(outer);